In [25]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



Spark NLP version 3.1.0
Apache Spark version: 3.0.2


In [26]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [27]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("pos"),
        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


+--------+---+---------+
|   token|pos|ner_label|
+--------+---+---------+
|    make|  0|      Act|
|     can|  0|      Act|
|      do|  0|      Act|
|    read|  0|      Act|
|   write|  0|      Act|
|    talk|  0|      Act|
|    want|  0|      Act|
|    cook|  0|      Act|
|    brew|  0|      Act|
|   clean|  0|      Act|
|   brush|  0|      Act|
|    care|  0|      Act|
|    walk|  0|      Act|
|      go|  0|      Act|
|navigate|  0|      Act|
|activity|  0|      Act|
|   write|  0|      Act|
|    move|  0|      Act|
|    fall|  0|      Act|
|      up|  0|      Act|
+--------+---+---------+
only showing top 20 rows



In [28]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [29]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)

embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(100)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [30]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[-1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-10-05 20:20:27.608530: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 7 - chars: 24 - training examples: 138
Epoch 1/100 started, lr: 0.003, dataset size: 138
Epoch 1/100 - 0.63s - loss: 32.87796 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.16s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 4	 0	 0.5	 1.0	 0.6666667
Act	 0	 1	 12	 0.0	 0.0	 0.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 16	 7	 5	 0.6956522	 0.7619048	 0.72727275
Obj	 17	 7	 1	 0.7083333	 0.9444444	 0.8095238
tp: 37 fp: 19 fn: 19 labels: 5
Macro-average	 prec: 0.3807971, rec: 0.54126984, f1: 0.44706947
Micro-average	 prec: 0.66071427, rec: 0.66071427, f1: 0.66071427
Epoch 2/100 started, lr: 0.0029850747, dataset size: 138
Epoch 2/100 - 0.07s - loss: 17.350958 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 1	 0	 3	 1.0	 0.25	 0.4
Act	 4	 6	 8	 0.4	 0.33333334	 0.36363637
Ne

Epoch 17/100 - 0.07s - loss: 0.6744518 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 1	 0	 0.8	 1.0	 0.88888896
Act	 7	 1	 5	 0.875	 0.5833333	 0.7
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 20	 1	 1	 0.95238096	 0.95238096	 0.95238096
Obj	 16	 5	 2	 0.7619048	 0.8888889	 0.82051283
Pos	 0	 1	 0	 0.0	 0.0	 0.0
tp: 47 fp: 9 fn: 9 labels: 6
Macro-average	 prec: 0.5648809, rec: 0.57076716, f1: 0.5678088
Micro-average	 prec: 0.83928573, rec: 0.83928573, f1: 0.83928573
Epoch 18/100 started, lr: 0.0027649768, dataset size: 138
Epoch 18/100 - 0.07s - loss: 0.57640314 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 1	 0	 0.8	 1.0	 0.88888896
Act	 7	 1	 5	 0.875	 0.5833333	 0.7
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 20	 1	 1	 0.95238096	 0.95238096	 0.95238096
Obj	 16	 5	 2	 0.7619048	 0.8888889	 0.82051283
Po

Epoch 32/100 - 0.07s - loss: 0.13202858 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 3	 0	 0.5714286	 1.0	 0.72727275
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 2	 0	 0.9130435	 1.0	 0.95454544
Obj	 13	 3	 5	 0.8125	 0.7222222	 0.76470584
Pos	 0	 1	 0	 0.0	 0.0	 0.0
tp: 46 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.53097683, rec: 0.5648148, f1: 0.54737335
Micro-average	 prec: 0.8214286, rec: 0.8214286, f1: 0.8214286
Epoch 33/100 started, lr: 0.002586207, dataset size: 138
Epoch 33/100 - 0.07s - loss: 0.09405151 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 3	 0	 0.5714286	 1.0	 0.72727275
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 2	 0	 0.9130435	 1.0	 0.95454544
Obj	 13	 3	 5	 0.8125	 0.7222222	 0.7

Epoch 47/100 - 0.07s - loss: 0.028917357 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 0	 0	 1.0	 1.0	 1.0
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 1	 0	 0.95454544	 1.0	 0.9767442
Obj	 17	 3	 1	 0.85	 0.9444444	 0.8947368
Pos	 0	 1	 0	 0.0	 0.0	 0.0
tp: 50 fp: 6 fn: 6 labels: 6
Macro-average	 prec: 0.6155724, rec: 0.6018519, f1: 0.6086348
Micro-average	 prec: 0.89285713, rec: 0.89285713, f1: 0.89285713
Epoch 48/100 started, lr: 0.0024291498, dataset size: 138
Epoch 48/100 - 0.07s - loss: 0.09953066 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 3	 0	 0.5714286	 1.0	 0.72727275
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 3	 0	 0.875	 1.0	 0.93333334
Obj	 12	 3	 6	 0.8	 0.6666667	 0.72727275
Pos	 0	 1	 0	 

Epoch 62/100 - 0.07s - loss: 0.030129218 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 2	 0	 0.6666667	 1.0	 0.8
Act	 6	 1	 6	 0.85714287	 0.5	 0.6315789
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 1	 0	 0.95454544	 1.0	 0.9767442
Obj	 15	 5	 3	 0.75	 0.8333333	 0.7894737
Pos	 0	 1	 0	 0.0	 0.0	 0.0
tp: 46 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.5380592, rec: 0.5555555, f1: 0.5466674
Micro-average	 prec: 0.8214286, rec: 0.8214286, f1: 0.8214286
Epoch 63/100 started, lr: 0.0022900763, dataset size: 138
Epoch 63/100 - 0.07s - loss: 0.015690902 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 2	 0	 0.6666667	 1.0	 0.8
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 1	 0	 0.95454544	 1.0	 0.9767442
Obj	 15	 3	 3	 0.8333333	 0.8333333	 0.8333333
Pos	 0	 1	 

Epoch 77/100 - 0.07s - loss: 0.0063983286 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 2	 0	 0.6666667	 1.0	 0.8
Act	 7	 1	 5	 0.875	 0.5833333	 0.7
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 2	 0	 0.9130435	 1.0	 0.95454544
Obj	 14	 4	 4	 0.7777778	 0.7777778	 0.77777773
Pos	 0	 1	 0	 0.0	 0.0	 0.0
tp: 46 fp: 10 fn: 10 labels: 6
Macro-average	 prec: 0.538748, rec: 0.5601852, f1: 0.5492575
Micro-average	 prec: 0.8214286, rec: 0.8214286, f1: 0.8214286
Epoch 78/100 started, lr: 0.002166065, dataset size: 138
Epoch 78/100 - 0.07s - loss: 0.013038116 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 2	 0	 0.6666667	 1.0	 0.8
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 2	 0	 0.9130435	 1.0	 0.95454544
Obj	 14	 3	 4	 0.8235294	 0.7777778	 0.8000001
Pos	 0	 1	 

Epoch 92/100 - 0.07s - loss: 0.006850895 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 2	 0	 0.6666667	 1.0	 0.8
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 2	 0	 0.9130435	 1.0	 0.95454544
Obj	 14	 3	 4	 0.8235294	 0.7777778	 0.8000001
Pos	 0	 1	 0	 0.0	 0.0	 0.0
tp: 47 fp: 9 fn: 9 labels: 6
Macro-average	 prec: 0.5486881, rec: 0.5740741, f1: 0.5610941
Micro-average	 prec: 0.83928573, rec: 0.83928573, f1: 0.83928573
Epoch 93/100 started, lr: 0.0020547945, dataset size: 138
Epoch 93/100 - 0.07s - loss: 0.026422733 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 4	 2	 0	 0.6666667	 1.0	 0.8
Act	 8	 1	 4	 0.8888889	 0.6666667	 0.7619048
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 21	 2	 0	 0.9130435	 1.0	 0.95454544
Obj	 14	 3	 4	 0.8235294	 0.7777778	 0.8000001


In [31]:
import pyspark.sql.functions as F
predictions = ner_model.transform(testing_data) 
predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
           .select(F.expr("cols['0']").alias("token"),
                   F.expr("cols['1']").alias("ground_truth"),
                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

2022-10-05 20:20:36.796372: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

+--------+------------+----------+
|token   |ground_truth|prediction|
+--------+------------+----------+
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|location|Loc         |Loc       |
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|kitchen |Loc         |Loc       |
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
+--------+------------+----------+
only showing top 20 rows



In [32]:
from sklearn.metrics import classification_report
 
preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()
 
print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       138
         Act       1.00      1.00      1.00        42
         Loc       1.00      1.00      1.00        36

    accuracy                           1.00       216
   macro avg       1.00      1.00      1.00       216
weighted avg       1.00      1.00      1.00       216



In [33]:
import shutil

shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

'/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en.zip'

In [34]:
text_list = [
    """Navagate to kitchen"""
]


In [44]:
from sparknlp.base import LightPipeline

light = LightPipeline(ner_model)
light.annotate("Navigate to kitchen")

{'embeddings': [], 'ner': [], 'ner_chunk': []}

In [52]:




from sparknlp_display import NerVisualizer



result = predictions

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)




